In [1]:
# import libraries
import gc
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from tqdm import tqdm
import lightning.pytorch as pl

gc.collect()
torch.cuda.empty_cache()

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

os.chdir('/home/nico/dev/projects/ai/musai')

sys.path.append('./src/tools')
sys.path.append('./src/model')

In [4]:
from embed import DEVICE, LR, DROPOUT, VAE

BATCH_SIZE = 4
X_DIM = 1024
EPOCHS = 3000
EPOCH_BEGIN = 199
EMBED_DIM = 1024
VOCAB_SIZE = 560
LATENT_DIM = 256
HIDDEN_DIM = 1024

PARAMS = {
    'batch': BATCH_SIZE,
    'x_dim': X_DIM,
    'embed_dim': EMBED_DIM,
    'hidden_dim': HIDDEN_DIM,
    'vocab_size': VOCAB_SIZE,
    'latent_dim': LATENT_DIM,
    'dropout': DROPOUT,
    'lr': LR
}

DEVICE

device(type='cuda')

In [5]:
# model = VAE(EMBED_DIM, LATENT_DIM, hidden_dims=[HIDDEN_DIM*4,
#             HIDDEN_DIM*2, HIDDEN_DIM, HIDDEN_DIM//2], vocab_size=560)
model = VAE.from_pretrained(
    f'out/all/embvae_{EPOCH_BEGIN}.pth',
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    latent_dim=LATENT_DIM,
    hidden_dim=HIDDEN_DIM
)

model


VAE(
  (emb): Embedding(560, 1024, padding_idx=0)
  (encoder): Encoder(
    (module): Sequential(
      (0): Linear(in_features=1024, out_features=4096, bias=False)
      (1): Sequential(
        (0): Linear(in_features=4096, out_features=2048, bias=False)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (2): Dropout(p=0.1, inplace=False)
      (3): Sequential(
        (0): Linear(in_features=2048, out_features=1024, bias=False)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (4): Sequential(
        (0): Linear(in_features=1024, out_features=512, bias=False)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (5): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=False)
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (fc_mean): Linear(in_features=256, out_features=256, bias=True)
    (fc_logvar): Linear(in_features=256, out_features=256, bias=True)
  )
  (decoder): Decoder(
    (module): Sequential(
      (0): Linear(in_features

In [6]:
from pathlib import Path
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import json

paths = list(
    Path('/home/nico/data/ai/models/midi/all/').glob('*_mid.json'))

tokens = []

for path in paths:
  tokens += json.load(open(path))['ids']

ids = torch.LongTensor(tokens)
ids = torch.split(ids, X_DIM)
ids = pad_sequence(ids, batch_first=True)
dataset = TensorDataset(ids)
data_loader = DataLoader(dataset, pin_memory=True, batch_size=BATCH_SIZE)

ids.shape


torch.Size([182551, 1024])

In [ ]:
from lightning.pytorch.callbacks import Callback
import datetime

EPOCHS_SAVE = 1


class TrainCallback(Callback):
    def on_train_epoch_start(self, trainer, pl_module):
        if trainer.is_global_zero:
            if trainer.global_step == 0:
                timestamp = datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
                log = open('./embvae-train_log.txt', 'a')

                log.write(f'NEW RUN {timestamp}\n{PARAMS}\n')

    def on_train_epoch_end(self, trainer, pl_module):
        if trainer.is_global_zero:  # logging & save state_dict
            if (trainer.current_epoch % EPOCHS_SAVE == 0):
                to_save_dict = pl_module.state_dict()

                try:
                    torch.save(
                        to_save_dict,
                        f'./embvae-{VOCAB_SIZE}x{EMBED_DIM}x{HIDDEN_DIM}-{EPOCH_BEGIN + 1 + trainer.current_epoch}.pth',
                    )
                except Exception as error:
                    print(error)

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx):
        if trainer.is_global_zero:
            param_groups = trainer.optimizers[0].param_groups
            lr = param_groups[-1]['lr']

    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        if trainer.is_global_zero:
            param_groups = trainer.optimizers[0].param_groups
            lr = param_groups[-1]['lr']


In [ ]:
trainer = pl.Trainer(
    devices='auto',
    max_epochs=100000,
    accelerator="auto",
    log_every_n_steps=100,
    callbacks=[
        TrainCallback()
    ],
    enable_checkpointing=False,
    limit_train_batches=3000
)
trainer.fit(model=model, train_dataloaders=data_loader)


In [8]:
import torch.nn.functional as F

model.to(DEVICE)
model.eval()

sample = dataset[np.random.randint(0, len(dataset)-1)][0]

with torch.no_grad():
    output, emb_hat, emb, hidden, mean, log_var = model(sample.to(DEVICE))

F.mse_loss(emb_hat, emb)

tensor(0.0109, device='cuda:0')